In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pillow_heif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.3 MB/s eta 0:00:00


In [ ]:
import glob
from torchvision.transforms.v2 import ToTensor
from torchvision.io import decode_image
from torchvision.transforms.v2.functional import to_image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os

from PIL import Image
import pillow_heif
pillow_heif.register_heif_opener()

data_dir = r'/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification'
image_lists = []

for subfolder in glob.glob(os.path.join(data_dir, '*', 'hand_written_digit', '??52????')):
    for num in range(10):
        pattern = os.path.join(subfolder, f'{num}_*.*')
        matched = glob.glob(pattern)
        image_lists.extend(matched)

print(f'FOUND {len(image_lists)} IMAGES.')


FOUND 7059 IMAGES.


In [ ]:
class custom_image_dataset(Dataset):
    def __init__(self, image_lists, transform=None):
        self.image_lists = image_lists
        self.transform = transform

    from PIL import Image

    def __read_image(self, img_path):
      try:
        image = Image.open(img_path).convert("RGB")  # đảm bảo 3 kênh RGB
        label = int(os.path.basename(img_path)[0])
        if self.transform:
            image = self.transform(image)
        return image, label
      except Exception as e:
          print(f"[ERROR] Cannot load image {img_path}: {e}")
          return  torch.zeros(3, 288, 288), 0 # None
    def __getitem__(self, idx):
        return self.__read_image(self.image_lists[idx])

    def __len__(self):
        return len(self.image_lists)

In [ ]:
import torchvision.transforms.v2 as transforms

train_transform = transforms.Compose([
    transforms.Resize((288, 288)),
    # transforms.RandomResizedCrop((288, 288), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True), # ToTensor
    transforms.Normalize(   [0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((288, 288)),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(   [0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
])

In [ ]:
split = int(0.9 * len(image_lists))
train_dataset = custom_image_dataset( image_lists[:split],
                                    transform=train_transform )
test_dataset = custom_image_dataset( image_lists[split:],
                                    transform=test_transform )

def safe_collate(batch):
    batch = [b for b in batch if b is not None]
    return torch.utils.data.dataloader.default_collate(batch)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, collate_fn=safe_collate)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, collate_fn=safe_collate)

In [ ]:
from torchvision import models
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

model = models.efficientnet_b2(weights='IMAGENET1K_V1')
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=num_ftrs, out_features=10)  # 10 digits
)
# model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for digits (0-9)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.0001,
    weight_decay=1e-4
)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Using cpu device


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 50.1MB/s]


In [ ]:
import time
import copy

def train_model(model, criterion, optimizer, scheduler,
                train_loader, test_loader, num_epochs=25, patience=2, save_path='best_model.pth'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    no_improve = 0  # Đếm số epoch không cải thiện

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            dataloader = train_loader if phase == 'train' else test_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Kiểm tra cải thiện trên tập validation
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    no_improve = 0  # reset lại
                    # Lưu model
                    torch.save(model.state_dict(), save_path)
                    print(f"Saved new best model with acc: {best_acc:.4f}")
                else:
                    no_improve += 1
                    print(f"No improvement for {no_improve} epoch(s).")

        # Dừng sớm nếu không cải thiện liên tục
        if no_improve >= patience:
            print(f"\nEarly stopping triggered after {patience} epoch(s) without improvement.")
            break

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model



In [ ]:
model_save_path = '/content/drive/MyDrive/effnet_b2.pth'
model = train_model(model, criterion, optimizer, scheduler,
                    train_loader, test_loader,
                    num_epochs=15, patience=2,
                    save_path=model_save_path)


Epoch 0/14
----------


100%|██████████| 100/100 [42:45<00:00, 25.66s/it]


train Loss: 1.7613 Acc: 0.4502


 67%|██████▋   | 8/12 [03:27<01:23, 20.80s/it]

[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/0_5.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/0_5.jpeg'
[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/0_6.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/0_6.jpeg'
[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/3_6.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/3_6.jpeg'


...

[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_1.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_1.jpeg'
[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_4.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_4.jpeg'
[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_9.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/6_9.jpeg'
[ERROR] Cannot load image /c

 83%|████████▎ | 10/12 [00:23<00:06,  3.44s/it]

[ERROR] Cannot load image /content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/8_1.jpeg: cannot identify image file '/content/drive/MyDrive/Teaching/CS114/CS114.P21/handwritten_digit_classification/NThong325_CS114/hand_written_digit/23521523/8_1.jpeg'


100%|██████████| 12/12 [00:29<00:00,  2.44s/it]

val Loss: 0.5963 Acc: 0.8314
No improvement for 2 epoch(s).

Early stopping triggered after 2 epoch(s) without improvement.

Training complete in 105m 35s
Best val Acc: 0.8371


# Eval

In [ ]:
unlabeled_dir = r'/content/drive/MyDrive/data.2025'
unlabeled_images = []

for root, dirs, files in os.walk(unlabeled_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.heic', '.jfif', '.md')):
            unlabeled_images.append(os.path.join(root, file))
        else:
            print(f"Skipped file: {file}")

print(f"Found {len(unlabeled_images)} images.")

Found 9998 images.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            if path.lower().endswith('.md'):
                placeholder = torch.zeros((3, 288, 288), dtype=torch.float32)
                return placeholder, os.path.basename(path)

            file_size = os.path.getsize(path)
            if file_size < 10:
                placeholder = torch.zeros((3, 288, 288), dtype=torch.float32)
                return placeholder, os.path.basename(path)

            else:
                image = Image.open(path).convert("RGB")
                image = to_image(image)
                if self.transform:
                    image = self.transform(image)
                return image, os.path.basename(path)

        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            placeholder = torch.zeros((3, 288, 288), dtype=torch.float32)
            return placeholder, os.path.basename(path)

In [ ]:
eval_dataset = CustomDataset(unlabeled_images, transform=test_transform)
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
from torchvision import models
import torch.nn as nn

model = models.efficientnet_b2(weights='IMAGENET1K_V1')
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=num_ftrs, out_features=10)  # 10 lớp (0-9)
)

# Load lại trọng số đã lưu
model.load_state_dict(torch.load('/content/drive/MyDrive/effnet_b2.pth', map_location=device))
model = model.to(device)

model.eval()
predictions = []
filenames = []

with torch.no_grad():
    for images, image_paths in tqdm(eval_loader):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        predictions.extend(preds.cpu().numpy())
        filenames.extend(image_paths)

  1%|▏         | 2/157 [03:36<3:58:43, 92.41s/it] 

In [ ]:
import pandas as pd

results = pd.DataFrame({
    'Filename': filenames,
    'Prediction': predictions
})

output_path = '/content/drive/MyDrive/predictions.csv'
results.to_csv(output_path, header=False, index=False)
print(f"Predictions saved to {output_path}")